In [41]:
import ctf,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()

In [64]:
def updateFactor(T,curr,others,regParam,omega,other_dimensions,r,s,idx):
    '''Update single factor matrix by using the formula'''
    
    other_dimensions_ = other_dimensions.copy()
    other_dimensions_.append(r)
    M = ctf.tensor((tuple(other_dimensions_)))
    
    # M.i("jku") << V.i("ju")*W.i("ku")
    S = s[:idx] + s[idx + 1:]
    temp = 1
    for k in range(len(others)):
        temp = temp*others[k].i(S[k]+'u')
    M.i(S+'u') << temp
    
    D = np.prod(other_dimensions)
    [U_,S_,V_] = ctf.svd(M.reshape((D,r)))
    S_ = 1./S_
    curr.set_zero()
    curr.i(s[idx]+'u') << V_.i("vu")*S_.i("v")*U_.reshape((tuple(other_dimensions_))).i(S+'v')*T.i(s)
    #U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
    normalize(curr)
    return curr

#def updateU(T,V,W,regParam,omega,I,J,K,r):
#    '''Update U matrix by using the formula'''
#    
#    M1 = ctf.tensor((J,K,r))
#    M1.i("jku") << V.i("ju")*W.i("ku")
#    [U_,S_,V_] = ctf.svd(M1.reshape((J*K,r)))
#    S_ = 1./S_
#    U.set_zero()
#    U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
#    normalize(U)
#    
#    return U      

In [59]:
def getALSCtf(T,factors,regParam,omega,dimensions,r):
    '''
    Tensor completion using ALS
    '''
    it = 0
    E = ctf.tensor(tuple(dimensions))
    curr_err_norm = 0
    
    import string
    s = string.ascii_lowercase
    s = s[:len(dimensions)]
    
    e = omega.i(s)
    for idx in range(len(factors)):
        print(s[idx]+'u')
        e = e*factors[idx].i(s[idx]+'u')
        #E.i("ijk") << T.i("ijk") - omega.i("ijk")*U.i("iu")*V.i("ju")*W.i("ku")
        curr_err_norm += ctf.vecnorm(factors[idx])     
    
    E.i(s) << T.i(s) - e 
    curr_err_norm = ctf.vecnorm(E) + curr_err_norm*regParam
    
    while True:
  #      U = updateU(T,V,W,regParam,omega,I,J,K,r) 
  #      V = updateV(T,U,W,regParam,omega,I,J,K,r) 
  #      W = updateW(T,U,V,regParam,omega,I,J,K,r) 
        for idx in range(len(factors)):
            others = factors.copy()
            others.pop(idx)
            other_dimensions = dimensions.copy()
            other_dimensions.pop(idx)
            factors[idx] = updateFactor(T,factors[idx],others,regParam,omega,other_dimensions,r,s,idx)
            
        next_err_norm = 0
        e = omega.i(s)
        for idx in range(len(factors)):
            e = e*factors[idx].i(s[idx]+'u')
            next_err_norm += ctf.vecnorm(factors[idx])      
        E.i(s) << T.i(s) - e 
        next_err_norm = ctf.vecnorm(E) + next_err_norm*regParam
  
        if curr_err_norm - next_err_norm > .001 or it > 10:
            break
         
        print(curr_err_norm, next_err_norm)
        curr_err_norm = next_err_norm
        it += 1
        
    return factors

In [60]:
def normalize(Z):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [61]:
def init_tensor(n):
    dimensions = []
    while n>0:
        r = random.randint(1,10)
        dimensions.append(r)
        n -= 1
    #print("dimensions: ", tuple(dimensions))
    T = ctf.tensor(tuple(dimensions),sp=True)
    T.fill_sp_random(0,1,1)
    return T,dimensions


def init_factors(dimensions,r):
    D = dimensions.copy()
    ctf.random.seed(42)
    factors = []
    while len(D) != 0:
        I = D.pop(0)
        U = ctf.random.random((I,r))
        normalize(U)
        factors.append(U)
    return factors


def updateOmega(dimensions,sparsity):
    '''
    Gets a random subset of rows
    '''
    Actf = ctf.tensor(tuple(dimensions),sp=True)
    Actf.fill_sp_random(1,1,sparsity)
    #omegactf = ((Actf > 0)*ctf.astensor(1.))
    return Actf

In [62]:
#n = random.randint(1,5) # arbitrary dimension
n = 3
r = 2 # TODO: determine rank?
sparsity = .2
regParam = 2
  
# arbitrary tensor
T,dimensions = init_tensor(n)
print("dimensions: ", tuple(dimensions))

factors = init_factors(dimensions,r)

omega = updateOmega(dimensions,sparsity)

dimensions:  (9, 1, 5)


In [65]:
getALSCtf(T,factors,regParam,omega,dimensions,r)

au
bu
cu
11.95345306394011 15.488194061072955
15.488194061072955 18.997335385403822
18.997335385403822 22.49656643239736
22.49656643239736 25.99506533728202
25.99506533728202 29.49267753849951
29.49267753849951 32.98854831800005
32.98854831800005 36.482149317482325
36.482149317482325 39.97330354655594
39.97330354655594 43.46209649838629
43.46209649838629 46.948777165416914
46.948777165416914 50.4336709085524


[array([[ 0.41489614, -0.40605474],
        [ 0.41797344, -0.33665655],
        [ 0.5248232 , -0.48950891],
        [ 0.42191552, -0.21904996],
        [ 0.11522394,  0.07977692],
        [ 0.17078664,  0.44362029],
        [ 0.09404685,  0.1341189 ],
        [ 0.34343449,  0.05515419],
        [ 0.17455832,  0.45824325]]),
 array([[ 1.,  1.]]),
 array([[ 0.39415176, -0.26206253],
        [ 0.63673764,  0.41161874],
        [ 0.52389358,  0.25040646],
        [ 0.19322016,  0.33717996],
        [ 0.35694686,  0.76517941]])]

### Tests

In [66]:
## TODO : write tests